In [1]:

from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [3]:

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp-25/02-experiment-tracking/mlruns/1', creation_time=1746490334307, experiment_id='1', last_update_time=1746490334307, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [4]:
client.create_experiment(name="my-cool-experiment")

'2'

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [9]:

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 037704f826ad45d4a9d1b0554746532c, rmse: 6.3141
run id: 5ffc32d1d86e4c7e805862db04ff036b, rmse: 6.3902
run id: d201213f07224203a9978aabaa54acb3, rmse: 6.6207
run id: 1da59f2ff1de447190cc92bad2687843, rmse: 6.6207
run id: c2dd5b9acf7d428681866e79869f8440, rmse: 6.6207


In [10]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [11]:
run_id = "037704f826ad45d4a9d1b0554746532c"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1746538389312, current_stage='None', description=None, last_updated_timestamp=1746538389312, name='nyc-taxi-regressor', run_id='037704f826ad45d4a9d1b0554746532c', run_link=None, source='/workspaces/mlops-zoomcamp-25/02-experiment-tracking/mlruns/1/037704f826ad45d4a9d1b0554746532c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [12]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None


/tmp/ipykernel_11594/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [31]:

model_version = 1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_11594/2494180744.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1746537644975, current_stage='Production', description='', last_updated_timestamp=1746539022287, name='nyc-taxi-regressor', run_id='d201213f07224203a9978aabaa54acb3', run_link='', source='/workspaces/mlops-zoomcamp-25/02-experiment-tracking/mlruns/1/d201213f07224203a9978aabaa54acb3/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost', 'validation_status': 'staging'}, user_id=None, version=1>

In [14]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)


<ModelVersion: aliases=[], creation_timestamp=1746538389312, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2025-05-06', last_updated_timestamp=1746538637642, name='nyc-taxi-regressor', run_id='037704f826ad45d4a9d1b0554746532c', run_link=None, source='/workspaces/mlops-zoomcamp-25/02-experiment-tracking/mlruns/1/037704f826ad45d4a9d1b0554746532c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [17]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [18]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")


In [20]:
run_id = "d201213f07224203a9978aabaa54acb3"
model_uri = f"runs:/{run_id}/model"

In [21]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/workspaces/mlops-zoomcamp-25/02-experiment-tracking/preprocessor'

In [22]:

import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [23]:
X_test = preprocess(df, dv)


In [24]:

target = "duration"
y_test = df[target].values


In [32]:
%time test_model(name=model_name, stage="Production",X_test=X_test, y_test=y_test)


CPU times: user 19.6 s, sys: 287 ms, total: 19.9 s
Wall time: 11.8 s


{'rmse': 6.525537591113061}

In [ ]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)


In [ ]:

client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)
